In [1]:
import mysql.connector
import pandas as pd

# Connection with the database
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="rootpassword",
  database="recetas_db"
)

# Load data from the database into dataframes
ingredients_df = pd.read_sql_query("SELECT * FROM ingredientes", conn)
recipes_ingredients_df = pd.read_sql_query("SELECT * FROM recetas_ingredientes", conn)
allergens_df = pd.read_sql_query("SELECT * FROM alergenos", conn)
restriccions_df = pd.read_sql_query("SELECT * FROM restricciones", conn)

# Load data from the database into dataframes
conn.close()

C:\Users\jcela\AppData\Local\Temp\ipykernel_15920\2738617342.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ingredients_df = pd.read_sql_query("SELECT * FROM ingredientes", conn)
C:\Users\jcela\AppData\Local\Temp\ipykernel_15920\2738617342.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recipes_ingredients_df = pd.read_sql_query("SELECT * FROM recetas_ingredientes", conn)
C:\Users\jcela\AppData\Local\Temp\ipykernel_15920\2738617342.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  allergens_df = pd.read_sql_qu

In [2]:
recipes_ingredients_df 

,ID,ID_RECETA,ID_INGREDIENTE,Cantidad,Grasa,Grasas_saturadas,Grasas_trans,Colesterol,Sodio,Carbohidratos,Proteina,Calorias
0,1,1,1,250.0,8.8,2.9,0.1,1.63,0.13,0.0,51.5,300.0
1,2,1,2,45.0,0.3,0.1,0.0,0.00,0.06,8.0,0.8,36.0
2,3,1,3,50.0,0.3,0.0,0.0,0.00,0.02,1.8,1.1,11.5
3,4,1,4,325.0,0.4,0.1,0.0,0.00,0.01,11.8,2.1,49.0
4,5,1,5,30.0,0.1,0.0,0.0,0.00,0.00,3.2,0.2,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
148,149,20,6,294.0,22.3,3.6,0.0,0.00,2.18,144.9,23.5,873.2
149,150,20,5,67.0,0.1,0.0,0.0,0.00,0.00,7.0,0.5,20.1
150,151,20,68,200.0,38.8,20.2,1.6,0.12,0.06,9.3,4.9,396.0
151,152,20,78,2.7,0.2,0.1,0.0,0.00,0.00,1.7,0.2,7.5


In [6]:
import pandas as pd
import numpy as np
import random
from typing import List
from dataclasses import dataclass
import mysql.connector

@dataclass
class Ingredient:
    id: int
    quantity: float
    total_fat: float 
    saturated_fats: float
    trans_fat: float
    cholesterol: float
    sodium: float
    carbohydrates: float
    protein: float
    calories: float

@dataclass
class Ant:
    recipe: List[Ingredient]

class AntColony:
    def __init__(self,
                 ingredients_df: pd.DataFrame,
                 recipes_ingredients_df: pd.DataFrame,
                 allergens_df: pd.DataFrame,
                 restriccions_df: pd.DataFrame,
                 hard_constraints: str,
                 num_ants: int = 10, 
                 evaporation_rate: float = 0.5,
                 alpha: int = 1,
                 beta: int = 2,
                 pheromone_init: float = 0.1,
                 pheromone_deposit: int = 1,
                 max_iterations: int = 100) -> None:
        self.ingredients_df = ingredients_df
        self.recipes_ingredients_df = recipes_ingredients_df
        self.allergens_df = allergens_df
        self.restriccions_df = restriccions_df
        self.hard_constraints = hard_constraints
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.pheromone_init = pheromone_init
        self.pheromone_deposit = pheromone_deposit
        self.max_iterations = max_iterations
        self.num_ingredients = len(self.ingredients_df)
        self.pheromone_matrix = np.full((self.num_ingredients, self.num_ingredients), pheromone_init)
        self.best_solution = None
        self.best_solution_fitness = float('inf')

    def _optimize(self) -> List[int]:
        for _ in range(self.max_iterations):
            solutions = self._construct_solution()
            for solution in solutions:
                solution_fitness = self.evaluate_solution_fitness(solution)
                if solution_fitness < self.best_solution_fitness:
                    self.best_solution = solution
                    self.best_solution_fitness = solution_fitness
            self.update_pheromone_matrix(solutions)
        return self.best_solution
    
    def _construct_solution(self) -> List[Ant]:
        ants = [Ant(recipe=[]) for _ in range(self.num_ants)]
        for ant in ants:
            recipe_length = random.randint(5, 15)
            selected_ingredients = set() # to avoid repeating ingredients
            while len(ant.recipe) < recipe_length:
                next_ingredient = self._select_next_ingredient(ant.recipe, selected_ingredients)
                if next_ingredient is not None:
                    ant.recipe.append(next_ingredient)
                    selected_ingredients.add(next_ingredient.id)
                else:
                    break  # No more valid ingredients
        return ants
    
    def _select_next_ingredient(self, current_recipe: List[Ingredient], selected_ingredients: set) -> Ingredient:
        probabilities = []
        available_ingredients = [i for i in range(1, self.num_ingredients + 1) if i not in selected_ingredients]

        print(f"Available ingredients before checking: {available_ingredients}")
        
        for ingredient_id in available_ingredients:
            if self._check_ingredient(ingredient_id):
                pheromone = sum(self.pheromone_matrix[ingredient_id - 1][i.id - 1] for i in current_recipe)
                desirability = self._calculate_desirability(ingredient_id)
                probabilities.append((ingredient_id, (pheromone ** self.alpha) * (desirability ** self.beta)))
        
        if not probabilities:
            return None
        
        total_probability = sum(prob for _, prob in probabilities)
        if total_probability == 0:
            return None
        probabilities = [(ing, prob / total_probability) for ing, prob in probabilities]
        selected_ingredient_id = random.choices([ing for ing, _ in probabilities], [prob for _, prob in probabilities])[0]
        selected_ingredient = self._create_ingredient(selected_ingredient_id)
        return selected_ingredient
    
    def _create_ingredient(self, ingredient_id: int) -> Ingredient:
        ingredient_data = self.recipes_ingredients_df[self.recipes_ingredients_df['ID_INGREDIENTE'] == ingredient_id].sample()
        ingredient_row = ingredient_data.iloc[0]
        return Ingredient(
            id=ingredient_id,
            quantity=ingredient_row['Cantidad'],
            total_fat=ingredient_row['Grasa'],
            saturated_fats=ingredient_row['Grasas_saturadas'],
            trans_fat=ingredient_row['Grasas_trans'],
            cholesterol=ingredient_row['Colesterol'],
            sodium=ingredient_row['Sodio'],
            carbohydrates=ingredient_row['Carbohidratos'],
            protein=ingredient_row['Proteina'],
            calories=ingredient_row['Calorias']
        )

    def _calculate_desirability(self, ingredient_id: int) -> float:
        return 1.0

    def _check_ingredient(self, ingredient_id: int) -> bool:
        constraints_list = self.hard_constraints.split(',')
        vegan = int(constraints_list[0])
        vegetarian = int(constraints_list[1])
        user_allergens = [int(x) for x in constraints_list[2:]]
        
        ingredient = self.ingredients_df[self.ingredients_df['ID'] == ingredient_id]
        if ingredient.empty:
            raise ValueError(f'Ingredient with id: {ingredient_id} not found')
        
        ingredient_vegan = ingredient['Vegano'].iloc[0]
        ingredient_vegetarian = ingredient['Vegetariano'].iloc[0]
        
        if vegan and not ingredient_vegan:
            return False
        if vegetarian and not ingredient_vegetarian:
            return False
        
        ingredient_allergens = self.restriccions_df[self.restriccions_df['ID_INGREDIENTE'] == ingredient_id]['ID_ALERGENO'].tolist()
        if set(user_allergens).intersection(set(ingredient_allergens)):
            return False
        
        return True
    
    def evaluate_solution_fitness(self, solution: Ant) -> float:
        total_nutrition = self._calculate_total_nutrition(solution.recipe)
        fitness = 0

        # Define the target ranges
        target_ranges = {
            'Grasa': (13, 27),
            'Grasas_saturadas': (5, 7),
            'Grasas_trans': (0, 0.3),
            'Colesterol': (0.05, 0.1),
            'Sodio': (0.5, 0.8),
            'Carbohidratos': (75, 108),
            'Proteina': (15, 35),
            'Calorias': (600, 700)
        }

        # Calculate the fitness score
        for nutrient, (min_val, max_val) in target_ranges.items():
            if total_nutrition[nutrient] < min_val:
                fitness += (min_val - total_nutrition[nutrient]) ** 2
            elif total_nutrition[nutrient] > max_val:
                fitness += (total_nutrition[nutrient] - max_val) ** 2

        return fitness
    
    def _calculate_total_nutrition(self, recipe: List[Ingredient]) -> pd.Series:
        total_nutrition = pd.Series(
            data={
                'Cantidad': 0,
                'Grasa': 0,
                'Grasas_saturadas': 0,
                'Grasas_trans': 0,
                'Colesterol': 0,
                'Sodio': 0,
                'Carbohidratos': 0,
                'Proteina': 0,
                'Calorias': 0
            }
        )
        for ingredient in recipe:
            total_nutrition += pd.Series(
                data={
                    'Cantidad': ingredient.quantity,
                    'Grasa': ingredient.total_fat,
                    'Grasas_saturadas': ingredient.saturated_fats,
                    'Grasas_trans': ingredient.trans_fat,
                    'Colesterol': ingredient.cholesterol,
                    'Sodio': ingredient.sodium,
                    'Carbohidratos': ingredient.carbohydrates,
                    'Proteina': ingredient.protein,
                    'Calorias': ingredient.calories
                }
            )
        return total_nutrition
    
    def update_pheromone_matrix(self, solutions: List[Ant]) -> None:
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        for ant in solutions:
            fitness = self.evaluate_solution_fitness(ant)
            if fitness == 0:
                fitness = 1  # Avoid division by zero
            pheromone_contribution = self.pheromone_deposit / fitness
            for i in range(len(ant.recipe) - 1):
                self.pheromone_matrix[ant.recipe[i].id -1][ant.recipe[i+1].id - 1] += pheromone_contribution

if __name__ == "__main__":
    hard_constraints = '0,0,1'  # This can be adjusted as needed
    ant_colony = AntColony(
        ingredients_df=ingredients_df, 
        recipes_ingredients_df=recipes_ingredients_df, 
        allergens_df=allergens_df, 
        restriccions_df=restriccions_df, 
        hard_constraints=hard_constraints
    )
    best_recipe = ant_colony._optimize()
    for ingredient in best_recipe.recipe:
        print(f'''ID: {ingredient.id},
              Quantity: {ingredient.quantity}, 
              Total Fat: {ingredient.total_fat}, Saturated Fats: {ingredient.saturated_fats},
              Trans Fat: {ingredient.trans_fat}, Cholesterol: {ingredient.cholesterol},
              Sodium: {ingredient.sodium}, Carbohydrates: {ingredient.carbohydrates},
              Protein: {ingredient.protein}, Calories: {ingredient.calories}''')

Available ingredients before checking: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78]
Available ingredients before checking: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78]
Available ingredients before checking: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,